# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [85]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [86]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# Tokenize and Stemming


In [87]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [88]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

stemming 시작
stemming 완료


# Garbage Collection 1

In [93]:
import gc

'''del [[mbti]]

del sentence

del [[dataCombine]]

del [condition]

for ownsentence in allData:
    del ownsentence
del allData

for tokenData in tokenizedData:   # delete 'stemData'
    for data in tokenData:
        del data
    del tokenData
del tokenizedData'''

for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text

del X_data

del tokenData
del tempData


gc.collect()

765

# Vectorization (tfidf)

In [94]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

X = tfidf.fit_transform(stemData)
print(X)

print(y_data)
y = np.array(y_data)
print(y)

  (0, 215484)	0.07970397738849286
  (0, 135346)	0.04572473782029354
  (0, 205310)	0.03950418216234991
  (0, 158065)	0.04066858473970089
  (0, 23848)	0.0109935806265749
  (0, 141006)	0.011381994850527067
  (0, 278900)	0.040253402609101166
  (0, 105270)	0.014199540818957095
  (0, 128750)	0.061839845362681736
  (0, 186259)	0.010992291523457435
  (0, 80119)	0.0391929541257891
  (0, 84095)	0.03344810938677439
  (0, 288699)	0.018554642544113317
  (0, 218864)	0.013689340850182625
  (0, 134021)	0.0612064668042313
  (0, 226365)	0.039820552926778705
  (0, 57549)	0.0399161899873013
  (0, 249687)	0.015142040472794304
  (0, 45592)	0.022988301510518117
  (0, 291298)	0.015917293926600354
  (0, 10629)	0.04979351788638793
  (0, 252187)	0.012140043789887327
  (0, 93076)	0.013783196037802266
  (0, 162449)	0.012433911245716332
  (0, 241971)	0.01857817436817611
  :	:
  (8674, 128827)	0.07222971717357766
  (8674, 171377)	0.01377446594629055
  (8674, 93612)	0.006625261778918338
  (8674, 62031)	0.030993570193

In [95]:
print(tfidf_matrix.shape)

max_words = max(len(l) for l in X)   # post의 최대 길이(단어 수)
print(max_words)
'''print('post의 평균 길이 :{}'.format(sum(map(len, stemData))/len(stemData)))
print('post의 전체 길이 :{}'.format(sum(map(len, stemData))))'''
post_num = len(stemData)    #post의 갯수
print(post_num)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

# Garbage Collection

In [72]:
import psutil
# import os
# import gc

In [73]:
for data in stemData:   # delete 'stemData'
    for component in data:
        del component
    del data
del stemData

gc.collect()

144

In [76]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)

model = Sequential()
model.add(Embedding(max_words, post_num))
model.add(LSTM(128))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split = 0.2)

NameError: name 'X' is not defined

In [ ]:
model.save('.\\models\\LSTM_undersample.h5')

In [ ]:
model.summary()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=32)